In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

import time
import math
import random
from tqdm import tqdm
import regex as re
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

## Identify sequences that have always patterns and those who don't

In [2]:
always_patterns = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\EDA\Data\always_patterns_8_1.csv")

In [3]:
always_pattern_regex = always_patterns["Pattern"].to_list()

In [4]:
for i in range(len(always_pattern_regex)):
    always_pattern_regex[i] = re.compile(always_pattern_regex[i], re.IGNORECASE)

In [6]:
train_df = pd.read_csv("Storage/Data/input.csv") 

train_df = train_df[['Unnamed: 0', 'patient_id', 'sequence','original', 'label']]
#train_df.columns = ['Unnamed: 0', 'patient_id', 'sequence','original', 'annotator_label']
train_df.head()

,Unnamed: 0,patient_id,sequence,original,label
0,0,Z15564314,s other free text-see phs viewer social histor...,s other free text-see phs viewer Social Histor...,1
1,1,Z10171706,------- fusion: no sleep disturbance: no socia...,------- fusion: No Sleep disturbance: No Socia...,1
2,2,Z8935348,ain spasm). 30 tablet 0 unknown (outside pharm...,ain/spasm). 30 tablet 0 Unknown (outside pharm...,1
3,3,Z12212893,------- on 112mcg dose. maria will check dose ...,------- on 112mcg dose. Maria will check dos...,2
4,4,Z9598376,------- (vibramycin) 100 mg capsule take 1 cap...,------- (VIBRAMYCIN) 100 MG capsule Take 1 ca...,1


In [10]:
a = []
c = []
counter = 0

for seq in tqdm(train_df["original"]):
    curr = []
    classes = []
    conversion = {2: "Yes", 1 : "Neither", 0: "No"}
    
    for p in (always_pattern_regex):
        m = list(set(re.findall(p, seq)))
        m = list(set(map(str.lower, m)))
        if (m != []):
            curr.append("".join(m))
            label = int(train_df.loc[counter]['label'])
            classes.append(conversion[label])
            classes = list(set(classes))
            
    a.append(curr)
    c.append(classes)
    counter += 1

train_df["always_pattern_match"] = a
train_df["always_pattern_classes"] = c

100%|████████████████████████████████████████████████████████████████████████████| 8656/8656 [00:07<00:00, 1152.80it/s]


In [12]:
train_df.head()

,Unnamed: 0,patient_id,sequence,original,label,always_pattern_match,always_pattern_classes
0,0,Z15564314,s other free text-see phs viewer social histor...,s other free text-see phs viewer Social Histor...,1,"[father, uncle, their first cousin, and his ma...",[Neither]
1,1,Z10171706,------- fusion: no sleep disturbance: no socia...,------- fusion: No Sleep disturbance: No Socia...,1,"[father with alzheimer's disease, father with ...",[Neither]
2,2,Z8935348,ain spasm). 30 tablet 0 unknown (outside pharm...,ain/spasm). 30 tablet 0 Unknown (outside pharm...,1,[family history: family history problem relati...,[Neither]
3,3,Z12212893,------- on 112mcg dose. maria will check dose ...,------- on 112mcg dose. Maria will check dos...,2,[cognitive deficits],[Yes]
4,4,Z9598376,------- (vibramycin) 100 mg capsule take 1 cap...,------- (VIBRAMYCIN) 100 MG capsule Take 1 ca...,1,[father history social history • marital statu...,[Neither]


In [14]:
# get sequences that were manually reviewed
manual_review = train_df[train_df["always_pattern_classes"].str.len() == 0]
manual_review.head()

,Unnamed: 0,patient_id,sequence,original,label,always_pattern_match,always_pattern_classes
75,75,Z10504958,ndings: general: no apparent distress. surgica...,ndings: General: No apparent distress. Surgica...,0,[],[]
77,77,Z6411816,"edness, no syncope, no dyspnea endocrine no po...","edness, No Syncope, No Dyspnea Endocrine No Po...",2,[],[]
87,87,Z6813204,5 reps calibrated hand gripper with silver spr...,5 reps Calibrated hand gripper with silver sp...,0,[],[]
95,95,Z10492528,nancial management services: none healthcare p...,nancial management Services: none Healthcare p...,0,[],[]
106,106,Z7656517,2 | 1 time: 1 | 0 time: 0) 3. “are there any r...,2 | 1 time: 1 | 0 time: 0) 3. “Are there any r...,0,[],[]


In [18]:
# get sequences that were annotated by always patterns 
always_patterns = train_df.drop(manual_review.index)
len(always_patterns)

8050

In [23]:
further_review = train_df["further_review"] = np.where((len(train_df["always_pattern_classes"]) > 1), 'N', 'Y')
further_review = train_df[train_df["further_review"] == "Y"]

In [24]:
further_review

,Unnamed: 0,patient_id,sequence,original,label,always_pattern_match,always_pattern_classes,further_review


In [25]:
manual_review.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Modeling\Storage\Data\manual_review.csv", index = False)

In [26]:
always_patterns.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Modeling\Storage\Data\always_patterns.csv", index = False)

## Clean 20K Sample

In [27]:
df = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\Diversity_Sampling\20K_sample_8_9.csv")

In [28]:
def clean_sequence(seq):
    #getting rid of special characters
    specials = '/' #etc
    seq_no_special_chars = seq.translate(str.maketrans(specials, ' '*len(specials)))
    
    #having only 1 space between words
    n = 1
    seq_no_spaces = (' '*n).join(seq_no_special_chars.split())
    
    return seq_no_spaces.lower()

In [29]:
for i in range(len(df)):
    df.loc[i, "regex_sent"] = clean_sequence(df.loc[i]["regex_sent"][7:len(df.loc[i]["regex_sent"]) - 5])

In [31]:
df.head()

,index,PatientID,MRN,EMPI,PatientEncounterID,InpatientNoteTypeDSC,NoteID,NoteCSNID,ContactDTS,NoteTXT,regex_sent,regex_match,in_diverse_df
0,47,Z6450279,10041636548,100051335,3.091909e+09,Op Note,628032759,6.184841e+08,2015-09-21,----------------------------------------------...,cystectomy due to his recurrent non- muscle in...,"[""dementia""]",NaN
1,163,Z6450779,10041647644,100051533,3.106767e+09,Progress Notes,1178659451,1.123313e+09,2016-08-10,----------------------------------------------...,ghter continues on treatment for metastatic co...,"[""dementia""]",NaN
2,338,Z6453461,10041699397,100052395,3.117106e+09,Consults,1003261998,9.576252e+08,2016-04-29,----------------------------------------------...,6 neurosurgery attending: dr. cahill dr. anoop...,"[""dementia""]",NaN
3,367,Z6453666,5757087,100052451,3.134078e+09,Progress Notes,1329295911,1.268146e+09,2016-11-22,----------------------------------------------...,egative. neurological: negative. hematological...,"[""dementia""]",NaN
4,426,Z6454616,10041719567,100052715,3.222378e+09,Progress Notes,2241215445,2.206259e+09,2018-10-22,----------------------------------------------...,"with an fi of 0.53, likely with diminished cog...","[""dementia"", ""MCI""]",NaN


In [34]:
df = df[['PatientID', 'regex_sent']]

In [35]:
df.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Modeling\Storage\Data\20K_sample.csv", index = False)

## Select new sample cohort (all sequences for all patients except those that were included in inital 8656)

In [45]:
entire = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\nlp_annotator-dev\app\load_data\SLAT_production_7_24.csv")

In [46]:
len(entire), len(train_df)

(279224, 8656)

In [47]:
patient_id_included = train_df["patient_id"].unique().tolist()

In [48]:
entire["include"] = np.where(entire["PatientID"].isin(patient_id_included), 'Y', 'N')

In [49]:
new_sample = entire[entire["include"] == "N"]

In [51]:
len(new_sample["PatientID"].unique().tolist()), len(new_sample)

(13941, 186730)

In [54]:
new_sample

,PatientID,MRN,EMPI,PatientEncounterID,InpatientNoteTypeDSC,NoteID,NoteCSNID,ContactDTS,NoteTXT,regex_sent,regex_match,include
0,Z6352398,10040002098,100000272,3.242118e+09,Progress Notes,2370049295,2.338373e+09,2019-03-14,----------------------------------------------...,"""\""ession alone in the meta-analysis. We discu...","[""dementia""]",N
2,Z6353136,10027033512,100001150,3.018789e+09,Discharge Summary,267681869,2.555255e+08,2015-01-09,----------------------------------------------...,"""\"" ------- l obstruction ASSOCIATED DIAGNOSES...","[""Dementia"", ""cognitive impairment""]",N
3,Z6353461,10040011735,100001521,3.172504e+09,Progress Notes,1682366001,1.631850e+09,2017-10-03,----------------------------------------------...,"""\"" ------- (156 lb) 09/19/17 71.4 kg (157 lb...","[""dementia"", ""Cognition""]",N
4,Z6353764,10040014499,100001873,3.165983e+09,Progress Notes,1853759390,1.807703e+09,2018-02-07,----------------------------------------------...,"""\""ut answer. Optho notes from recent outpatie...","[""dementia""]",N
5,Z6354111,969233,100002288,3.327443e+09,Progress Notes,4616537753,4.605039e+09,2020-09-22,----------------------------------------------...,"""\""hat would be more consistent. Has been aler...","[""dementia""]",N
...,...,...,...,...,...,...,...,...,...,...,...,...
279219,Z16320629,10127549029,113705759,3.285425e+09,Progress Notes,3960511043,3.943607e+09,2020-04-28,----------------------------------------------...,"""\""''she took a break' and wants to discuss wi...","[""memory""]",N
279220,Z16320629,10127549029,113705759,3.305416e+09,Progress Notes,4585859437,4.574134e+09,2020-09-22,----------------------------------------------...,"""\""'all of her medications, including aspirin....","[""memory""]",N
279221,Z16320629,10127549029,113705759,3.327430e+09,Progress Notes,5203067949,5.224024e+09,2021-01-25,----------------------------------------------...,"""\""'Patient, Chart, daughter Olga Mode of cont...","[""cognition""]",N
279222,Z16320629,10127549029,113705759,3.349755e+09,Progress Notes,5673786544,5.670392e+09,2021-04-20,----------------------------------------------...,"""\""'Patient, Chart, daughter Olga Mode of cont...","[""cognition""]",N


In [53]:
new_sample.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Sampling\sample_8_12.csv", index = False)